In [ ]:
!nvidia-smi

Sun Dec 27 01:23:58 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
os.chdir('/content/drive/MyDrive/AOI') #切換該目錄
os.listdir() #確認目錄內容

['test_images.zip',
 'classify',
 'Transfer_Training_Report.ipynb',
 'Transfer_Training_models',
 'Transfer_Training.h5',
 'Transfer_Training.ipynb',
 'Model.h5',
 'Model.ipynb',
 'Dir_info.ipynb',
 'test_images',
 'csv',
 'Model_Predict.ipynb',
 'Quality_Control.ipynb',
 'Transfer_Training_Predict.ipynb']

# load model

In [ ]:
from keras.models import load_model

# 載入模型
model = load_model('Model.h5')

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# data flow and predict

In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input

test_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=preprocess_input
)

test_itr = test_datagen.flow_from_directory(
    './test_images',
    target_size=(256, 256),
    batch_size=512,
    class_mode='categorical',
    shuffle=False
)

Found 0 images belonging to 1 classes.


In [ ]:
pred = model.predict_generator(test_itr, verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


38/38 [==============================] - 3891s 105s/step


NameError: ignored

In [ ]:
import numpy as np

# 输出每个图像的预测类别
predicted_class_indices = np.argmax(pred, axis=1)

In [ ]:
import pandas as pd

dict_pred = { i: 0  for i in range(6)}
for i in predicted_class_indices:
    dict_pred[i] += 1

df_pred = pd.DataFrame([dict_pred])
df_pred

,0,1,2,3,4,5
0,6425,3468,639,2793,1136,4817


# to csv

In [ ]:
import pandas as pd

df_test = pd.read_csv('./csv/test.csv')

In [ ]:
id = df_test.iloc[:, 0]

submission = pd.DataFrame({
    'ID': id,
    'Labels': predicted_class_indices
    })

submission.to_csv('./csv/submission.csv', index=False)
submission

,ID,Labels
0,test_00001.jpg,3
1,test_00002.jpg,4
2,test_00003.jpg,5
3,test_00004.jpg,5
4,test_00005.jpg,0
...,...,...
19273,test_19274.jpg,0
19274,test_19275.jpg,0
19275,test_19276.jpg,0
19276,test_19277.jpg,1
